In [32]:
import numpy as np

X = []
y = []

with open('train.dat', 'r') as f:
    for line in f:
        _y, _X = line.split('\t')
        y.append(int(_y == '1'))
        X.append(_X[:-1])
X = np.array(X)
y = np.array(y)
X.shape

def tknzr(x):
    x_1 = [x[i] for i in range(len(x))]
    x_2 = [x[i]+x[i+1] for i in range(len(x)-1)]
    x_3 = [x[i]+x[i+1]+x[i+2] for i in range(len(x)-2)]
    x_4 = [x[i]+x[i+1]+x[i+2]+x[i+3] for i in range(len(x)-3)]
    xx = x_3 + x_4 + x_2
    return xx

testo = []
with open('test.dat', 'r') as f:
    for line in f:
        testo.append(line[:-1])
testo = np.array(testo)
v_set = np.append(testo, X)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier as abc
from scipy.sparse import csr_matrix

vectorizer = CountVectorizer(tokenizer=tknzr)

vectorizer.fit(v_set)

np.random.seed(10)

X_vec = vectorizer.transform(X)

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 40)
split = skf.split(X_vec, y)
X_vec_arr = X_vec.toarray()

mm = []
for train, test in split:
    mm.append((train, test))

# for m in range(1):
#     mc = 0
#     for k, (train, test) in enumerate(mm):
#         X_vec_train = []
#         y_train = []
#         for i in train:
#             X_vec_train.append(X_vec_arr[i])
#             y_train.append(y[i])
#         X_vec_test = []
#         y_test = []
#         for i in test:
#             X_vec_test.append(X_vec_arr[i])
#             y_test.append(y[i])
#         X_vec_test = np.array(X_vec_test)
#         y_test = np.array(y_test)
#         X_vec_train = np.array(X_vec_train)
#         y_train = np.array(y_train)
#         X_train_vec = csr_matrix(X_vec_train)
#         X_test_vec = csr_matrix(X_vec_test)

#         svc = abc(random_state = 30, n_estimators = i*10)
# #         svc = SVC(C=10, kernel='poly', degree=2, random_state=30, coef0 = 2.5, tol = 0.001, probability=True)
#         svc.fit(X_train_vec, y_train)
#         pred = svc.predict(X_test_vec)
# #         print(k, mcc(y_test, pred))
#         mc += mcc(y_test, pred)
#     print(m, mc/5)
    
# X_train_vec, X_test_vec, y_train, y_test = train_test_split(X_vec, y, test_size = 0.3, random_state = 60, stratify = [0, 1])

/WAVE/users/unix/sli13/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
for k, (train, test) in enumerate(mm):
    X_vec_train = []
    y_train = []
    for i in train:
        X_vec_train.append(X_vec_arr[i])
        y_train.append(y[i])
    X_vec_test = []
    y_test = []
    for i in test:
        X_vec_test.append(X_vec_arr[i])
        y_test.append(y[i])
    X_vec_test = np.array(X_vec_test)
    y_test = np.array(y_test)
    X_vec_train = np.array(X_vec_train)
    y_train = np.array(y_train)
    X_train_vec = csr_matrix(X_vec_train)
    X_test_vec = csr_matrix(X_vec_test)
    svc = abc(random_state = 30, n_estimators = 50, learning_rate = 0.1, algorithm='SAMME.R')
    svc.fit(X_train_vec, y_train)
    pred = svc.predict(X_test_vec)
    print(k,mcc(y_test, pred))

0 3.987761659531656e-01
1 4.064929878424972e-01
2 3.1385076732017947e-01
3 4.4601539744761465e-01
4 4.37488466817384e-01


In [6]:
svc = SVC(C=10, kernel='poly', degree=2, random_state=30, coef0 = 2.3, tol = 0.0001, probability=True)
svc.fit(X_vec, y)

testo_vec = vectorizer.transform(testo)

pred = svc.predict(testo_vec)


with open('outto3.dat', 'w') as f:
    for i in pred:
        f.write(str(1 if int(i+0.5)==1 else -1)+'\n')

In [23]:
import tensorflow as tf

from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

from gensim.corpora import Dictionary

X_tokens = [tknzr(x) for x in X]
vocab = Dictionary(X_tokens)
X_seq = [vocab.doc2idx(x) for x in X_tokens]

In [54]:
X_seq = pad_sequences(X_seq)
X_seq.shape

(1566, 129)

In [110]:
model = models.Sequential()
model.add(layers.Embedding(input_dim = len(vocab)+1, output_dim = 2, input_length = 129, mask_zero = True))
model.add(layers.Conv1D(256, 7, activation='relu'))
model.add(layers.MaxPool1D(pool_size=96, padding='valid'))
model.add(layers.Dense(1, activation='sigmoid'))

In [111]:
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['mse'])

In [112]:
np.random.seed(10)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size = 0.3, random_state = 10)
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
y  = np.asarray(y).astype('float32').reshape((-1,1))

In [113]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
35/35 [==============================] - 1s 7ms/step - loss: 0.5747 - mse: 0.1918
Epoch 2/20
35/35 [==============================] - 0s 7ms/step - loss: 0.3500 - mse: 0.0960
Epoch 3/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2907 - mse: 0.0790
Epoch 4/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2831 - mse: 0.0777
Epoch 5/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2713 - mse: 0.0758
Epoch 6/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2562 - mse: 0.0736
Epoch 7/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2381 - mse: 0.0708
Epoch 8/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2195 - mse: 0.0677
Epoch 9/20
35/35 [==============================] - 0s 7ms/step - loss: 0.2044 - mse: 0.0650
Epoch 10/20
35/35 [==============================] - 0s 7ms/step - loss: 0.1919 - mse: 0.0629
Epoch 11/20
35/35 [==============================] - 0s 7ms/step - lo

In [114]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 2ms/step - loss: 0.2583 - mse: 0.0764


[0.2582729160785675, 0.07642751187086105]

In [115]:
# from sklearn.svm import SVC

# svc = SVC(random_state=42, C=100, kernel='linear')

# svc.fit(X_train_vec, y_train)

# pred = svc.predict(X_test_vec)

# # mcc(pred, y_test)

# X_vec = vectorizer.transform(X)
# svc.fit(X_vec, y)

In [116]:
model.fit(X_seq, y)

49/49 [==============================] - 0s 7ms/step - loss: 0.1967 - mse: 0.0645


In [119]:
testo = []
with open('test.dat', 'r') as f:
    for line in f:
        testo.append(line[:-1])
testo = np.array(testo)
testo_tokens = np.array([tknzr(x) for x in testo])
testo_seq = np.array([vocab.doc2idx(x, unknown_word_index = 0) for x in testo_tokens])
testo_seq = pad_sequences(testo_seq)

p = model.predict(testo_seq)

with open('outto1.dat', 'w') as f:
    for i in p:
        f.write(str(1 if int(i+0.5)==1 else -1)+'\n')

/local/scratch/65422/ipykernel_109913/620370005.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  testo_tokens = np.array([tknzr(x) for x in testo])
/local/scratch/65422/ipykernel_109913/620370005.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  testo_seq = np.array([vocab.doc2idx(x, unknown_word_index = 0) for x in testo_tokens])


In [50]:
# import tensorflow as tf

# def to_sparse_tensor(X):
#     coo = X.tocoo()
#     indices = np.mat([coo.row, coo.col]).transpose()
#     return tf.SparseTensor(indices, coo.data, coo.shape)

2022-05-10 15:22:50.262029: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-10 15:22:50.262093: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [49]:
# to_sparse_tensor(X_train_vec)

NameError: name 'tf' is not defined